<a id="project2">
### Predict Crypto Currency Price Movement from Reddit Sentiment  
</a>

**Overview**: 

Using reddit posts under a given crypto currency's subreddit I want to predict the probability that the price of the currency will spike the following day. 

**Data Source**: [Poloniex Exchange](https://poloniex.com/support/api/) and [Reddit](https://bigquery.cloud.google.com/results/omega-wind-88718:bquijob_2add0008_15797586a12)

In [2]:
import json
import time
import requests
%matplotlib inline
import pylab
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
def get_poloniex_data(command, args={}):
    url = 'https://poloniex.com/public?command='+command
    for arg, value in args.items():
        url += '&{}={}'.format(arg,value)
    return json.loads(requests.get(url).content.decode('utf-8'))

Select the markets with the highest daily volume

In [4]:
market_volumes = get_poloniex_data('return24hVolume')
market_volumes = [(m, float(market_volumes[m]['BTC'])) for m in market_volumes if m[:3] == 'BTC']
markets = [m[0] for m in sorted(market_volumes, key=lambda k: k[1], reverse=True)[:20]]
markets

['BTC_ETH',
 'BTC_REP',
 'BTC_FCT',
 'BTC_XMR',
 'BTC_XRP',
 'BTC_ETC',
 'BTC_NXT',
 'BTC_CGA',
 'BTC_AMP',
 'BTC_DASH',
 'BTC_STEEM',
 'BTC_MAID',
 'BTC_SC',
 'BTC_SDC',
 'BTC_1CR',
 'BTC_LBC',
 'BTC_DOGE',
 'BTC_STR',
 'BTC_NAUT',
 'BTC_LTC']

For each market collect trade data for past day

In [15]:
def unix_time(dt):
    return int(time.mktime(dt.timetuple()))

def get_trade_history(currency_pair, start, end, save_file=False, file_path='data/'):
    date_fmt = '%Y-%m-%dT%H:%M:%S'
    file_name = file_path+'Trades|{}|{}|{}.csv'.format(
        currency_pair, start.strftime(date_fmt), end.strftime(date_fmt))
    
    trades = pd.DataFrame()
    need_to_fetch = lambda t: len(t) == 0 or len(t) % 50000 == 0
    while need_to_fetch(trades):
        print('fetching trades...')
        new_trades = pd.DataFrame(
            get_poloniex_data('returnTradeHistory', {
                'currencyPair': currency_pair,
                'start': unix_time(start),
                'end': unix_time(end if len(trades) == 0 
                                 else datetime.utcfromtimestamp(
                            trades.date.tail(1).values[0].tolist()/1e9))
            }))
        
        if new_trades.empty:
            break
        
        for col in ['amount','rate','total']:
            new_trades[col] = new_trades[col].astype(float)
        new_trades['date'] = pd.to_datetime(new_trades.date)
        
        if save_file:
            with open(file_name, 'a') as f:
                new_trades.to_csv(f, index=False)
                trades = new_trades
        else:
            trades = pd.concat([trades, new_trades])
        
        time.sleep(2)
    
    if save_file:
        return file_name

    return trades   

In [16]:
file_paths = {}

for market in markets:
    print(market.replace('_',':'))
    file_paths[market] = get_trade_history(market, datetime(2016,10,2), datetime(2016,10,3), save_file=True)

BTC:ETH
fetching trades...
BTC:REP
fetching trades...
BTC:FCT
fetching trades...
BTC:XMR
fetching trades...
BTC:XRP
fetching trades...
BTC:ETC
fetching trades...
BTC:NXT
fetching trades...
BTC:CGA
fetching trades...
BTC:AMP
fetching trades...
BTC:DASH
fetching trades...
BTC:STEEM
fetching trades...
BTC:MAID
fetching trades...
BTC:SC
fetching trades...
BTC:SDC
fetching trades...
BTC:1CR
fetching trades...
BTC:LBC
fetching trades...
BTC:DOGE
fetching trades...
BTC:STR
fetching trades...
BTC:NAUT
fetching trades...
BTC:LTC
fetching trades...


In [91]:
xmr_trades = pd.read_csv(file_paths['BTC_XMR'])
# add columns 
def ceil_dt(dt, delta):
    dt = datetime.strptime(dt,'%Y-%m-%d %H:%M:%S')
    return dt + (datetime.min - dt) % delta

def get_volume_level(total):
    level = 'very low'
    if 0.2 <= total <= 1:
        level = 'low'
    elif 1 <= total <= 5:
        level = 'medium'
    elif 5 <= total <= 10:
        level = 'high'
    elif 10 <= total:
        level = 'very high'
    return level
        

xmr_trades['time_bucket'] = xmr_trades.date.apply(lambda dt: ceil_dt(dt, timedelta(minutes=15)))   
xmr_trades['time'] = xmr_trades.date.apply(lambda dt: datetime.strptime(dt,'%Y-%m-%d %H:%M:%S'))
xmr_trades['volume_level'] = xmr_trades.total.apply(get_volume_level)
xmr_trades = xmr_trades[['time','type','rate','total', 'volume_level','time_bucket']].set_index('time')
xmr_trades.head()

,type,rate,total,volume_level,time_bucket
time,,,,,
2016-10-02 23:59:52,sell,0.013562,0.077188,very low,2016-10-03
2016-10-02 23:59:16,sell,0.013562,0.111030,very low,2016-10-03
2016-10-02 23:59:11,buy,0.013600,0.000000,very low,2016-10-03
2016-10-02 23:59:10,buy,0.013600,0.000130,very low,2016-10-03
2016-10-02 23:59:05,buy,0.013600,0.000000,very low,2016-10-03


In [105]:
grouped_trades = xmr_trades.groupby(['time_bucket','type','volume_level'])

In [142]:
stats = grouped_trades.describe().unstack(['type','volume_level',3])
stats

rate                                          \
type                     buy                                           
volume_level        very low                                           
                       count      mean       std       min       25%   
time_bucket                                                            
2016-10-02 00:00:00        1  0.012010       NaN  0.012010  0.012010   
2016-10-02 00:15:00       96  0.011965  0.000023  0.011931  0.011948   
2016-10-02 00:30:00      159  0.011993  0.000055  0.011917  0.011930   
2016-10-02 00:45:00      367  0.012258  0.000093  0.012060  0.012180   
2016-10-02 01:00:00      192  0.012311  0.000037  0.012205  0.012300   
2016-10-02 01:15:00      120  0.012230  0.000036  0.012138  0.012205   
2016-10-02 01:30:00      185  0.012342  0.000075  0.012195  0.012285   
2016-10-02 01:45:00      311  0.012564  0.000085  0.012304  0.012508   
2016-10-02 02:00:00       86  0.012496  0.000048  0.012390  0.012472   
2016-10-02 02:15:00       48  0.012406  0.000029  0.012360  0.012392   
2016-10-02 02:30:00       68  0.012461  0.000054  0.012365  0.012417   
2016-10-02 02:45:00       61  0.012595  0.000045  0.012525  0.012550   
2016-10-02 03:00:00       38  0.012470  0.000034  0.012405  0.012448   
2016-10-02 03:15:00       40  0.012328  0.000024  0.012276  0.012307   
2016-10-02 03:30:00       62  0.012290  0.000052  0.012142  0.012235   
2016-10-02 03:45:00       28  0.012304  0.000018  0.012280  0.012285   
2016-10-02 04:00:00      132  0.012439  0.000107  0.012285  0.012330   
2016-10-02 04:15:00       47  0.012468  0.000046  0.012400  0.012427   
2016-10-02 04:30:00       25  0.012463  0.000028  0.012427  0.012427   
2016-10-02 04:45:00       25  0.012498  0.000011  0.012487  0.012489   
2016-10-02 05:00:00       26  0.012442  0.000030  0.012400  0.012430   
2016-10-02 05:15:00       41  0.012295  0.000062  0.012248  0.012249   
2016-10-02 05:30:00      114  0.012421  0.000069  0.012269  0.012381   
2016-10-02 05:45:00       55  0.012408  0.000042  0.012350  0.012385   
2016-10-02 06:00:00       45  0.012515  0.000024  0.012470  0.012500   
2016-10-02 06:15:00       17  0.012547  0.000007  0.012538  0.012540   
2016-10-02 06:30:00       77  0.012563  0.000035  0.012460  0.012548   
2016-10-02 06:45:00       46  0.012556  0.000012  0.012519  0.012554   
2016-10-02 07:00:00       31  0.012512  0.000009  0.012493  0.012505   
2016-10-02 07:15:00       32  0.012476  0.000047  0.012430  0.012430   
...                      ...       ...       ...       ...       ...   
2016-10-02 16:45:00      438  0.013820  0.000115  0.013562  0.013740   
2016-10-02 17:00:00      235  0.013844  0.000075  0.013719  0.013800   
2016-10-02 17:15:00      190  0.013654  0.000054  0.013560  0.013616   
2016-10-02 17:30:00      239  0.013523  0.000028  0.013489  0.013500   
2016-10-02 17:45:00      167  0.013574  0.000027  0.013522  0.013555   
2016-10-02 18:00:00      228  0.013730  0.000068  0.013551  0.013690   
2016-10-02 18:15:00      136  0.013616  0.000047  0.013544  0.013566   
2016-10-02 18:30:00      186  0.013505  0.000097  0.013330  0.013415   
2016-10-02 18:45:00      118  0.013491  0.000065  0.013350  0.013439   
2016-10-02 19:00:00       99  0.013455  0.000038  0.013405  0.013425   
2016-10-02 19:15:00      120  0.013432  0.000049  0.013320  0.013403   
2016-10-02 19:30:00      161  0.013383  0.000042  0.013300  0.013378   
2016-10-02 19:45:00      197  0.013440  0.000060  0.013344  0.013384   
2016-10-02 20:00:00      193  0.013528  0.000012  0.013495  0.013520   
2016-10-02 20:15:00      201  0.013557  0.000017  0.013510  0.013545   
2016-10-02 20:30:00      119  0.013465  0.000053  0.013350  0.013410   
2016-10-02 20:45:00      189  0.013463  0.000075  0.013310  0.013410   
2016-10-02 21:00:00       83  0.013516  0.000034  0.013459  0.013485   
2016-10-02 21:15:00       64  0.013459  0.000006  0.013445  0.013456   
2016-10-02 21:30:00       62  0.013436  0.000018  0.013380  0.01

In [149]:
results = []

for i, r in stats.fillna(0).iterrows():
    result = {'date': i}
    for I1 in r.index.get_level_values(0).unique():
        for I2 in r.index.get_level_values(1).unique():
            for I3 in r.index.get_level_values(2).unique():
                result[I1+'_'+I2+'_'+I3] = r[I1,I2,I3]['mean']

    results.append(result)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/kernelbase.py:370: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


In [157]:
trades = pd.DataFrame(results).set_index('date')
high_vol = trades[[col for col in trades.columns if 'high' in col]]

In [158]:
high_vol.head()

,rate_buy_high,rate_buy_very high,rate_sell_high,rate_sell_very high,total_buy_high,total_buy_very high,total_sell_high,total_sell_very high
date,,,,,,,,
2016-10-02 00:00:00,0.000000,0.00000,0.00000,0,0.00000,0.000000,0.000000,0
2016-10-02 00:15:00,0.000000,0.00000,0.00000,0,0.00000,0.000000,0.000000,0
2016-10-02 00:30:00,0.000000,0.00000,0.00000,0,0.00000,0.000000,0.000000,0
2016-10-02 00:45:00,0.012288,0.01226,0.01218,0,6.07751,10.650957,5.634839,0
2016-10-02 01:00:00,0.000000,0.00000,0.00000,0,0.00000,0.000000,0.000000,0


In [116]:
trades.date.tail(1).values[0], trades.date.head(1).values[0]

(numpy.datetime64('2016-08-10T00:00:13.000000000+0000'),
 numpy.datetime64('2016-09-10T23:53:33.000000000+0000'))

In [76]:
trades = pd.DataFrame(trades)
for col in ['amount','rate','total']:
    trades[col] = trades[col].astype(float)
trades['date'] = pd.to_datetime(trades.date)
type(trades.date.values[0])

numpy.datetime64

In [31]:
datetime.min - np

datetime.datetime(1, 1, 1, 0, 0)

In [68]:
import asyncio
from os import environ
from autobahn.asyncio.wamp import ApplicationSession, ApplicationRunner


class Component(ApplicationSession):
    """
    An application component that subscribes and receives events, and
    stop after having received 5 events.
    """

    async def onJoin(self, details):

        self.received = 0

        def on_event(i):
            print("Got event: {}".format(i))
            self.received += 1
            if self.received > 5:
                self.leave()

        await self.subscribe(on_event, u'trollbox')

    def onDisconnect(self):
        asyncio.get_event_loop().stop()


if __name__ == '__main__':
    runner = ApplicationRunner(
        environ.get("AUTOBAHN_DEMO_ROUTER", u"wss://api.poloniex.com"),
        u"crossbardemo",
    )
    runner.run(Component)

ImportError: No module named 'autobahn'

In [159]:
!git add .
!git commit -m 'acquired trade data from poloniex'
!git push origin project

ExceptionPexpect: pty.fork() failed: [Errno 12] Cannot allocate memory